In [1]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.invoke("how many letters in the word educa?")
response

### Parse email dataset

In [ ]:
import pandas as pd  
# you can download from https://www.kaggle.com/datasets/wcukierski/enron-email-dataset/
df = pd.read_csv('./emails.csv',nrows=10)

In [ ]:
from apihelper.parsers import email_parser
with open(f"../../data/emails.txt","w") as fd:
    for item in df['message']:
        parsed_item,isok = email_parser(item)
        if isok:
            fd.write(f"{parsed_item}\n\n")

### Download and parse chat dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("FunDialogues/customer-service-robot-support", split="train")

In [ ]:
from apihelper.parsers import chat_parser
with open(f"../../data/chats.txt","w") as fd:
    for item in dataset['dialogue'][:10]:
        parsed_item,isok = chat_parser(item)
        if isok:
            fd.write(f"{parsed_item}\n\n")

### Quadrant test

In [2]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant

In [3]:
loader = TextLoader("../../data/emails.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1780, which is longer than the specified 500


In [4]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="../../data/quadrant_emails",
    collection_name="emails",
)

In [5]:
query = "how do you turn this on"
found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=2)

In [10]:
query = "how do you turn this on"
foo = qdrant.similarity_search_with_score(query,score_threshold=.2,k=5)

In [14]:
obj = foo[0]


In [17]:
obj[0].page_content

"I don't think these are required by the ISP \n\n  2.  static IP address\n\n IP: 64.216.90.105\n Sub: 255.255.255.248\n gate: 64.216.90.110\n DNS: 151.164.1.8\n\n  3.  Company: 0413\n        RC:  105891"